# Pangeo: Lunch and Learn Advanced Demo

## Section 1: Working with Dataframes
***
- [1.1 Parsing Landsat Metadata](#1.1)
- [1.2 Seaching Landsat 8 Scenes in the cloud with sat-search](#1.2)

## Section 2: Working with Landsatdata in the cloud
***
- [2.1 Draw Full Extent Image](#2.1)
- [2.2 Make thumbnail three band composites for two acqusitions](#2.2)
- [2.3 Make a full resolution windowed composite for two acqusitions](#2.3)
- [2.4 Calculating Normalized Burn Ratio](#2.4)
- [2.5 Creating a histogram of dNBR vaules](#2.5)
- [2.6 Masking classes dNBR vaules to show only burned areas](#2.6)
- [2.7 Creating and exporting custom layouts](#2.7)

### 1.1 Parsing Landsat Metadata <a id="1.1"></a>

Abstract: This tutorial will read a CSV formatted Landsat Metadata export retrieved from EarthExplorer and load the CSV file into a Pandas Dataframe. This allows for simple parsing of the CSV by specifying specific columns that contain your information of interest. With Pandas you can also export a new CSV containing only the columns required.

In [ ]:
#import required modules
import pandas as pd
import csv

In [ ]:
# Takes select columns from the CSV
df = pd.read_csv('LANDSAT_8_C1_314611.csv', sep = ',', usecols=['Landsat Product Identifier','Collection Category','WRS Path', 'WRS Row', 'Scene Cloud Cover'], squeeze=True)


In [ ]:
#Prints the Dataframe
df

In [ ]:
#Select scenes where the scene cloud score is between 0 and 10
df_cc = df[df['Scene Cloud Cover'].between(0, 10)]

In [ ]:
#Select the Landsat Product IDs that have meet the cloud score criteria specified above
df_productid = df_cc[['Landsat Product Identifier']]

In [ ]:
#Print the new dataframe which will contain only Product IDs meeting the cloud score criteria
df_productid

In [ ]:
# Exports the variable to a CSV file stripping the index numbers and the header information
df_productid.to_csv(r'example.csv', index=False, header=False)

### 1.2 Seaching Landsat 8 Scenes in the cloud with sat-search<a id="1.2"></a>
Abstract: This tutorial demonstrates querying Landsat 8 Data stored in the the cloud using the Sat-Search Python Module. Additional information on this library can be found here: https://github.com/sat-utils/sat-search. Currently this module is querying Landsat 8 imagery processed into cloud otimized format (COG, https://www.cogeo.org/) by Planet (https://www.planet.com/pulse/planet-labs-celebrates-landsat-8-data-is-finally-available-on-amazon-web-services/) although the intent is to hook our data to this library at somepoint. The purpose of this tutorial is how we invision our users searching the USGS Landsat Record in the cloud.

This tutorial introduces a new metadata format, Spatio Temporal Asset Catalog (STAC https://stacspec.org/). Each scene in the query's metadata is returned by the query in a machine readable JSON format. Importantly each component of the legacy TAR GZ file is stored uncompressed in the cloud. A link to these objects is prodvided as part of the JSON repsonse when the query is returned. This allows direct access to the imagery without specifying a direct URL to the object.

In this example Sat-Search is queried for Landsat scenes meeting certain requirements. The JSON is added to a Pandas dataframe and be using the object URL fo each scenes Band 4 a preview of Band 4 is drawn in the notebook.

In [ ]:
#Import modules
from satsearch import Search
import json
import geopandas as gpd
import rasterio
from rasterio.plot import show
import pandas as pd

In [ ]:
#Specify query parameters
#Search Extent Defined using Bounding Box: -97.50, 45.48, -97.21, 45.24
#Acquired April 15, 2019 to August 29, 2019
#Cloud Score less than 30
#Tier 1
#Landsat 8 Collection
results = Search.search(bbox=[-97.50, 45.48, -97.21, 45.24],
               datetime='2019-04-15/2019-08-29',
               property=["eo:cloud_cover<30", "landsat:tier=T1"],
               collection='landsat-8-l1')

In [ ]:
#Print Number of Scenes Meeting Criteria
print('%s items' % results.found())

In [ ]:
#Save Returned JSON Response for later use
items = results.items()
print(items)
items.save('searchResults.json')
jsonData = 'searchResults.json'

In [ ]:
#open JSON results
with open(jsonData) as f:
    json_results = json.load(f)

In [ ]:
#Helpful tip: With JSON results you can print the keys to help locating the fields you want in your dataframe later by using the command json_results.keys()
json_results.keys()

In [ ]:
#Print JSON for demonstration purposes.
print(json.dumps(json_results, indent = 4))

In [ ]:
# Load results to pandas geodataframe
#Sort by date acquired
gf = gpd.read_file('searchResults.json')
gf = gf.sort_values('datetime').reset_index(drop=True)
#print number of records as a check to verify number of scenes in the frame match the number returned by query
print('records:', len(gf))
gf.head(200)

In [ ]:
#Convert to Pandas Dataframe and display product ID, acquisition date, cloud cover, tier, column (path) and row
df = pd.DataFrame(gf, columns =['landsat:product_id','datetime','eo:cloud_cover','landsat:tier', 'eo:column', 'eo:row'])
#Print Dataframe
df

In [ ]:
#Write dataframe to CSV for later use
df.to_csv('results.csv', sep = ',')

In [ ]:
#Here the the JSON is parsed to locate the object URLs for all 12 canidate scene's band 4, the URLS are then appended to a list
bands = []
for items in json_results['features']:
    band4 = items['assets']['B4']
    bands.append(band4['href'])

In [ ]:
#Loop throw list of bands and open and read each file and cretae a overview thumbnail to preview each acqusition's band 4
for item in bands:
    with rasterio.open(item) as src:
        oviews = src.overviews(1) 
        oview = oviews[-1] 
        band = [src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))]
        show(band)

## Section 2: Working with Landsat data in the cloud

In [ ]:
#import required modules
import sys
import requests
from rasterio.rio import insp
import rasterio
import rasterio.plot
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.windows import Window
from matplotlib.colors import ListedColormap, BoundaryNorm
from skimage import data, img_as_float
from skimage import exposure
from matplotlib.patches import Patch

<bold> Important: The next two cells will load the band objects needed for the rest of the tutorial. Ideally you would use the sat-search module to query the record instead of hard coding the S3 links. But, since this a dev environment the S3 links have to be hard coded as a limitation of the dev environment.

In [ ]:
#Import required bands from before the fire
date = '7-19-2018'
Bb5_file = 's3://dev-usgs-landsat/collection1/level2/albers/oli-tirs/2018/044/032/LC08_L1TP_044032_20180719_20180731_01_A1/LC08_L1TP_044032_20180719_20180731_01_A1_sr_band5.tif'
Bb6_file = 's3://dev-usgs-landsat/collection1/level2/albers/oli-tirs/2018/044/032/LC08_L1TP_044032_20180719_20180731_01_A1/LC08_L1TP_044032_20180719_20180731_01_A1_sr_band6.tif'
Bb7_file = 's3://dev-usgs-landsat/collection1/level2/albers/oli-tirs/2018/044/032/LC08_L1TP_044032_20180719_20180731_01_A1/LC08_L1TP_044032_20180719_20180731_01_A1_sr_band7.tif'

In [ ]:
#Import required bands from after the fire
date2 = '1-27-2019'
Ab5_file = 's3://dev-usgs-landsat/collection1/level2/albers/oli-tirs/2019/044/032/LC08_L1TP_044032_20190127_20190206_01_A1/LC08_L1TP_044032_20190127_20190206_01_A1_sr_band5.tif'
Ab6_file = 's3://dev-usgs-landsat/collection1/level2/albers/oli-tirs/2019/044/032/LC08_L1TP_044032_20190127_20190206_01_A1/LC08_L1TP_044032_20190127_20190206_01_A1_sr_band6.tif'
Ab7_file = 's3://dev-usgs-landsat/collection1/level2/albers/oli-tirs/2019/044/032/LC08_L1TP_044032_20190127_20190206_01_A1/LC08_L1TP_044032_20190127_20190206_01_A1_sr_band7.tif'

### 2.1 Draw Full Extent Image<a id="2.1"></a>
Abstract: This tutorial will demonstrate opening, reading, and drawing a full extent of the NIR band for scene LC08_L1TP_044032_20190127_20190206_01_A1.

In [ ]:
with rasterio.open(Ab5_file) as src:
    AnirF = src.read(1)

In [ ]:
#Equalize histogram to increase brightness
AnirFeq = exposure.equalize_hist(AnirF)

#Plot full extent band 5
plt.figure(figsize=(40,40))
plt.imshow(AnirFeq, vmin = 0, vmax = 1, cmap = 'hot')
plt.colorbar(shrink=0.5)
plt.title('After Fire Band 5', fontsize = 20)
plt.xlabel('Column #', fontsize = 20)
plt.ylabel('Row #', fontsize = 20)

### 2.2 Make thumbnail three band composites for two acqusitions<a id="2.2"></a>
Abstract: This tutorial will demonstrate how to read three bands from two different acqusitions and draw a three band composite thumbnail for each acqusition in a SWIR2, SWIR, NIR band combination. The two scenes used as inputs are LC08_L1TP_044032_20180719_20180731_01_A1 and LC08_L1TP_044032_20190127_20190206_01_A1.    

In [ ]:
#Opens and reads each band file's URL
#Reduces to an overview low resolution thumbnail
with rasterio.open(Bb5_file) as src:
    oviews = src.overviews(1) 
    oview = oviews[-1] 
    BnirThumb = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
with rasterio.open(Bb6_file) as src:
    oviews = src.overviews(1) 
    oview = oviews[-1] # let's look at the smallest thumbnail
    BswirThumb = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
with rasterio.open(Bb7_file) as src:
    oviews = src.overviews(1) 
    oview = oviews[-1] 
    Bswir2Thumb = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

In [ ]:
#Opens and reads each band file's URL
#Reduces to an overview low resolution thumbnail
with rasterio.open(Ab5_file) as src:
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    oview = oviews[-1] # let's look at the smallest thumbnail
    AnirThumb = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
with rasterio.open(Ab6_file) as src:
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    oview = oviews[-1] # let's look at the smallest thumbnail
    AswirThumb = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
with rasterio.open(Ab7_file) as src:
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    oview = oviews[-1] # let's look at the smallest thumbnail
    Aswir2Thumb = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [ ]:
#Normalize thumbnail values to values between 0 and 1
Bswir2Thumbn = normalize(Bswir2Thumb)
BswirThumbn = normalize(BswirThumb)
BnirThumbn = normalize(BnirThumb)
Aswir2Thumbn = normalize(Aswir2Thumb)
AswirThumbn = normalize(AswirThumb)
AnirThumbn = normalize(AnirThumb)

In [ ]:
#Creates three band thumbnail composites using band combination 7, 6, 5
BcompThumb = np.dstack((Bswir2Thumbn, BswirThumbn, BnirThumbn))
AcompThumb = np.dstack((Aswir2Thumbn, AswirThumbn, AnirThumbn))

#Create Plot and display the thumbnail composites side by side
fig, axes = plt.subplots(1,2, figsize=(14,6), sharex=True, sharey=True)
plt.sca(axes[0])
plt.imshow(BcompThumb)
plt.colorbar(shrink=0.5)
plt.title('Before Fire Bands 7, 6, 5/ {}'.format(date))
plt.xlabel('Column #')
plt.ylabel('Row #')

plt.sca(axes[1])
plt.imshow(AcompThumb)
plt.colorbar(shrink=0.5)
plt.title('After Fire Band 7, 6, 5/ {}'.format(date2))
plt.xlabel('Column #')
plt.ylabel('Row #')

### 2.3 Make a full resolution windowed composite for two acqusitions<a id="2.3"></a>
Abstract: In this turtorial a windowed approach is used to create a full resolution three band composite image in a SWIR2, SWIR, NIR band combination. The two scenes used as inputs are LC08_L1TP_044032_20180719_20180731_01_A1 and LC08_L1TP_044032_20190127_20190206_01_A1. Processing is done only on the area of the scene within a specified area of interest or window. Instead of the window approach for rasterio can also support processing by GeoJSON or shapefile. https://rasterio.readthedocs.io/en/stable/index.html. 

In [ ]:
#Define window for future processing
window = Window.from_slices((5500, 6900), (2300, 3300))
#Opens and reads each band file's URL
#Create Windowed band array
#Repeat above for each band
with rasterio.open(Bb5_file) as src:
    Bnir = src.read(1, window=window)
with rasterio.open(Bb6_file) as src:
    Bswir = src.read(1, window=window)
with rasterio.open(Bb7_file) as src:
    Bswir2 = src.read(1, window=window)

In [ ]:
#Define window for future processing
window = Window.from_slices((5500, 6900), (2300, 3300))
#Opens and reads each band file's URL
#Create Windowed band array
#Repeat above for each band
with rasterio.open(Ab5_file) as src:
    Anir = src.read(1, window=window)
with rasterio.open(Ab6_file) as src:
    Aswir = src.read(1, window=window)
with rasterio.open(Ab7_file) as src:
    Aswir2 = src.read(1, window=window)

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [ ]:
#Normalize windowed bands to values randing from 0-1
Bswir2N = normalize(Bswir)
BswirN = normalize(Bswir)
BnirN = normalize(Bnir)
Aswir2N = normalize(Aswir2)
AswirN = normalize(Aswir)
AnirN = normalize(Anir)

In [ ]:
#Create three band raster composites 
#Equalize histagrams to increase brightness for display
Bswir2Neq = exposure.equalize_hist(Bswir2N)
BswirNeq = exposure.equalize_hist(BswirN)
BnirNeq = exposure.equalize_hist(BnirN)
Aswir2Neq = exposure.equalize_hist(Aswir2N)
AswirNeq = exposure.equalize_hist(AswirN)
AnirNeq = exposure.equalize_hist(AnirN)

#Create three band raster composites 
Bcomp = np.dstack((Bswir2Neq, BswirNeq, BnirNeq))
Acomp = np.dstack((Aswir2Neq, AswirNeq, AnirNeq))

#Plot raster composites side by side
fig, axes = plt.subplots(1,2, figsize=(40,40), sharex=True, sharey=True)
plt.sca(axes[0])
plt.imshow(Bcomp)
plt.title('Before Fire Bands 7, 6, 5/ {}'.format(date), fontsize=30)
plt.xlabel('Column #')
plt.ylabel('Row #')
plt.grid(b=None)
plt.axis('off')

plt.sca(axes[1])
plt.imshow(Acomp)
plt.title('After Fire Band 7, 6, 5/ {}'.format(date2), fontsize=30)
plt.axis('off')
plt.grid(b=None)

### 2.4 Calculating Normalized Burn Ratio<a id="2.4"></a>
Abstract: In this tutorial Normalized Burn Ration is calculated on two acqusitions based upon the extent of the windowed scenes created above. The input scenes are from before and after the Camp Fire (2018) near Paradise, California. Scene LC08_L1TP_044032_20180719_20180731_01_A1 is from before the fire and LC08_L1TP_044032_20190127_20190206_01_A1 is after the fire. In the final step of this section the dNBR (difference Normalized Burn Ratio) is calculated by subtracting the two scenes to create a burn intensity drawing.

In [ ]:
#Indicies (NBR) need to be calculate from integers, here arrays are converted to integers
Bswir2N = np.asarray(Bswir2, dtype = int)
BnirN = np.asarray(Bnir, dtype = int)
Aswir2N = np.asarray(Aswir2, dtype = int)
AnirN = np.asarray(Anir, dtype = int)

In [ ]:
#Mask Out Pixels that are 0
Bswir2Nmask = np.ma.masked_where(Bswir2N <=0, Bswir2N, copy=True)
BnirNmask = np.ma.masked_where(BnirN <=0, Bswir2N, copy=True)
Aswir2Nmask = np.ma.masked_where(BnirN <=0, Aswir2N, copy=True)
AnirNmask = np.ma.masked_where(BnirN <=0, AnirN, copy=True)

In [ ]:
#Define function to calculate NBR
def calcNBR(nir,swir2):
    nir = nir.astype('f4')
    swir2 = swir2.astype('f4')
    nbr = (nir - swir2) / (nir + swir2)
    return nbr

#Before Fire NBR Calculation
nbr = calcNBR(Bnir,Bswir2)

#After Fire NBR Calculation
nbr2 = calcNBR(Anir,Aswir2)

#Calculate dNBR
dnbr = (nbr-nbr2)

In [ ]:
#Plot dNBR
plt.figure(figsize=(40,40))
plt.imshow(dnbr, vmin = -1, vmax = 1, cmap = 'YlOrRd')
plt.colorbar(shrink=0.5)
plt.title('difference Normalized Burn Ratio Calculuation', fontsize=30)
plt.axis('off')

### 2.5 Creating a histogram of dNBR vaules<a id="2.5"></a>

Abstract: This section will plot a histogram of dNBR values for reference dNBR values are binned as follows:

< -0.25, High post-fire regrowth, -0.25 to -0.1  Low post-fire regrowth, -0.1 to +0.1 Unburned, 
0.1 to 0.27, Low-severity burn, 0.27 to 0.44, Moderate-low severity burn, 0.44 to 0.66 Moderate-high severity burn, 
< 0.66 High-severity burn.

In [ ]:
#Plot histogram of dNBR values with custom breaks using break values as specified above
bins = [-1, -.25, -.1, .1, .27, .44, .66, 1]
fig, ax = plt.subplots(figsize=(12,12))
ax.hist(dnbr.ravel(), color='blue', 
        edgecolor='white', 
        bins=bins)
ax.set(title="Histogram with Custom Breaks",
       xlabel="dBNR Value", ylabel="Number of Pixels")
ax.set_xticks(bins)

### 2.6 Masking classes dNBR vaules to show only burned areas <a id="2.6"></a>
Abstract: In this tutorial the classes containing areas with a dNBR value of less than .1 are masked to leave only classes containing burned areas.

In [ ]:
#This converts the bin values to a class
dnbrClass = np.digitize(dnbr, bins)

In [ ]:
#Show number of dBNR classes
print(np.unique(dnbrClass))

In [ ]:
#Here we will plot areas of low, moderate and high burn intensity so the first 3 clases are masked
dnbrClassmask = np.ma.masked_where(dnbrClass <=3, dnbrClass, copy=True)

In [ ]:
#Print masked array
dnbrClassmask

In [ ]:
#Insp Masked array to verify classes 1, 2, 3 have been masked
print(np.unique(dnbrClassmask))

### 2.7 Creating and exporting custom layouts <a id="2.7"></a>
Abstract: In this tutorial a custom layout is created showing areas with low, moderate and high burn intensity. For display purposes the scene from before the fire is used as a basemap with the final burn intensity placed over. The final layout can also be exported as a TIF for later use.

In [ ]:
#Define color ramp for classes
colors = ['Green','Yellow', 'Red']

#Define Labels for Legend
fireLabels = ['Low',"Moderate", "High"]

#Create Legend
patches = [Patch(color=icolor, label=label)
                  for icolor, label in zip(colors, fireLabels)]

#Assign Color Camp to cmap parameter
cmap = ListedColormap(colors)

#Create Plot Size
fig, ax = plt.subplots(figsize=(40, 20))
ax.set(title="Burn Intensity of Camp Fire (2018)")

#Show three class (low, moderate, high dNBR plot)
ax.imshow(dnbrClassmask, cmap=cmap)

#Show basemap with transparency
ax.imshow(Bcomp, alpha = .2)

#Show Legend
ax.legend(handles=patches, title_fontsize = '20', title = 'Burn Intensity',
         facecolor ="white",
         edgecolor = "white",
         loc = 'upper right',
         frameon=True,
         prop=dict(size=18)) 

#Turn off axis
ax.set_axis_off();

In [ ]:
fig.savefig('plot.tif')